In [1]:
import warnings
warnings.filterwarnings('ignore')
import netCDF4 as nc
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import rioxarray as rxr
import seaborn as sns
import geopandas as gpd
import earthpy as et

fn = './data_daily/*.nc4'

In [2]:
ds = xr.open_mfdataset(fn)
ds

,Array,Chunk
Bytes,153.89 MB,98.21 kB
Shape,"(1567, 248, 99)","(1, 248, 99)"
Count,4701 Tasks,1567 Chunks
Type,float32,numpy.ndarray


In [3]:
print(ds.variables.keys())

KeysView(Frozen({'precipitationCal': <xarray.Variable (time: 1567, lon: 248, lat: 99)>
dask.array<concatenate, shape=(1567, 248, 99), dtype=float32, chunksize=(1, 248, 99), chunktype=numpy.ndarray>
Attributes:
    units:         mm
    long_name:     Daily accumulated precipitation (combined microwave-IR) es...
    origname:      precipitationCal
    fullnamepath:  /precipitationCal, 'lat': <xarray.IndexVariable 'lat' (lat: 99)>
array([34.850002, 34.95    , 35.05    , 35.149998, 35.250004, 35.350002,
       35.45    , 35.55    , 35.649998, 35.750004, 35.850002, 35.95    ,
       36.05    , 36.149998, 36.250004, 36.350002, 36.45    , 36.55    ,
       36.649998, 36.750004, 36.850002, 36.95    , 37.05    , 37.149998,
       37.250004, 37.350002, 37.45    , 37.55    , 37.649998, 37.750004,
       37.850002, 37.95    , 38.05    , 38.150005, 38.249996, 38.350002,
       38.45001 , 38.55    , 38.650005, 38.749996, 38.850002, 38.95001 ,
       39.05    , 39.150005, 39.249996, 39.350002, 39.45

In [4]:


# Create a spatial map of your selected location with cartopy

# Set the spatial extent to cover the CONUS (Continental United States)
extent = [36, 42, 26, 45]
central_lon = np.mean(extent[:2])
central_lat = np.mean(extent[2:])

# Create your figure and axis object
# Albers equal area is a common CRS used to make maps of the United States
f, ax = plt.subplots(figsize=(12, 6),
                     subplot_kw={'projection': ccrs.AlbersEqualArea(central_lon, central_lat)})
ax.coastlines()
# Plot the selected location
ax.plot(longitude-360, latitude,
        '*',
        transform=ccrs.PlateCarree(),
        color="purple",
        markersize=10)

ax.set_extent(extent)
ax.set(title="Location of the Latitude / Longitude Being Used To to Slice Your netcdf Climate Data File")

# Adds continent boundaries to the map
ax.add_feature(cfeature.LAND, edgecolor='black')

ax.gridlines()
plt.show()

The min and max latitude values in the data is: 34.850002 44.650005
The min and max longitude values in the data is: 22.450008 47.150005


In [ ]:
ds.precipitationCal[0].plot()

In [ ]:
ds.precipitationCal[1].plot()

In [ ]:
ds.precipitationCal[2].plot()

In [ ]:
print("The min and max precipitation value at 20000601 is (in mm):",
      ds.precipitationCal[0].values.min(),
      ds.precipitationCal[0].values.max())

In [ ]:
print("The min and max precipitation value at 20000602 is (in mm):",
      ds.precipitationCal[1].values.min(),
      ds.precipitationCal[1].values.max())


In [ ]:
max_in_2000 = ds.precipitationCal.sel(time=slice('2000-01-01', '2000-12-31')).values.max()
print("The max precipitation value at 2000 is (in mm):",
      max_in_2000)

In [ ]:
avg_in_2000 = ds.precipitationCal.sel(time=slice('2000-01-01', '2000-12-31')).values.mean()
print("The average precipitation value at 2000 is (in mm):",
      avg_in_2000)

In [ ]:
max_in_2001 = ds.precipitationCal.sel(time=slice('2001-01-01', '2001-12-31')).values.max()
print("The max precipitation value at 2001 is (in mm):",
      max_in_2001)

In [ ]:
avg_in_2001 = ds.precipitationCal.sel(time=slice('2001-01-01', '2001-12-31')).values.mean()
print("The average precipitation value at 2001 is (in mm):",
      avg_in_2001)

In [ ]:
max_in_2002 = ds.precipitationCal.sel(time=slice('2002-01-01', '2002-12-31')).values.max()
print("The max precipitation value at 2002 is (in mm):",
      max_in_2002)

In [ ]:
avg_in_2002 = ds.precipitationCal.sel(time=slice('2002-01-01', '2002-12-31')).values.mean()
print("The average precipitation value at 2002 is (in mm):",
      avg_in_2002)


In [ ]:
ds_d_area_mean = ds.precipitationCal.mean(dim=('lon','lat'))

In [ ]:
gb_d = ds_d_area_mean.groupby('time.month')
ds_d_anom = gb_d - gb_d.mean(dim='time')
ds_d_anom_rl = ds_d_anom.rolling(time=6, center=True).mean()

fig,ax = plt.subplots(figsize=(16,9))
ds_d_anom_rl.plot(ax=ax, linewidth=1, color='k')

ax.set_xlabel('Time/year')
ax.set_title('Precipitaional Anomaly in Turkey',fontsize=20)
ax.set_ylabel('Anomaly')

plt.axhline(y=0.5,linestyle='-.',linewidth=1,color='blue')
plt.axhline(y=-0.5,linestyle='-.',linewidth=1,color='red')
plt.axhline(y=0,linewidth=0.5,color='k')

ax.set_xlim(['2000','2003'])
